<a href="https://colab.research.google.com/github/Akash743/Mathematical-Optimization/blob/main/Optimizer_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils  #glpk is the solver
!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
!unzip -o -q couenne-linux64
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

from pyomo.environ import * # for using pyomo functions like exp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 4.1 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155685 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Pr

In [53]:
model = pyo.ConcreteModel()

# Sets
model.i = pyo.RangeSet(1,6)
model.j = pyo.RangeSet(1,13)

In [56]:
x[i].pprint()

{Member of x} : Size=78, Index=x_index
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
    (6, 13) :     0 :   0.0 :  None : False : False : NonNegativeReals


In [16]:
import pandas as pd

model = pyo.ConcreteModel()

# Sets
model.i = pyo.RangeSet(1,6)
model.j = pyo.RangeSet(1,13)

# Parameters
S = pd.read_excel('Weekly_OPTIMIZATION_TESTING.xlsm',sheet_name='Spends',header=0,index_col=0,usecols='A:G',nrows=13)
C = pd.read_excel('Weekly_OPTIMIZATION_TESTING.xlsm',sheet_name='CPP',header=0,index_col=0,usecols='A:G',nrows=13)
T = pd.read_excel('Weekly_OPTIMIZATION_TESTING.xlsm',sheet_name='Transformation',header=0,index_col=0,usecols='A:H',nrows=13)

"""
i = M1, M2, M3...
j = W1, W2, W3...
"""

'\ni = M1, M2, M3...\nj = W1, W2, W3...\n'

In [17]:
C = C.replace(0,1)

T = T.fillna(0)

In [35]:
T.head(2)

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Adjustment_factor
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835,41767.022503
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069,151105.700000


In [125]:
S.head(2)

,1,2,3,4,5,6
Period,,,,,,
1,1681.5,0.0,6141.681550,0.0,0.000000,0.0
2,2476.1,0.0,15080.853226,12833.3,1547.846154,0.0


In [18]:
spends = {}
for i in range(1,len(S.columns)+1):
  for j in range(1,len(S.index)+1):
    spends[(i,j)] = S.loc[j,i]
cpp = {} 
for i in range(1,len(C.columns)+1):
  for j in range(1,len(C.index)+1):
    cpp[(i,j)] = C.loc[j,i]

In [5]:
T.head()

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Adjustment_factor
1.0,Media,1,382843.625,0.7,0.3,0.000835,41767.022503
2.0,Media,0.5,8401198.4375,0.8,0.5,0.000069,151105.7
3.0,Media,1.5,30349509.010398,0.5,0.4,0.000006,3617750
4.0,Media,3,113528318.317627,0.9,0.6,0.000022,510454.3
5.0,Promo,0,7274.307692,0,0,0.045272,0


In [57]:
S.head()

,1,2,3,4,5,6
Period,,,,,,
1,1681.50,0.0,6141.681550,0.00,0.000000,0.000000
2,2476.10,0.0,15080.853226,12833.30,1547.846154,0.000000
3,3279.79,0.0,7192.357366,4191.13,1665.318681,39133.014118
4,2585.39,0.0,3574.155648,1806.90,1712.307692,39133.014118
5,1051.12,0.0,0.000000,800.00,1824.175824,5590.430588


In [19]:
T['Ret_rate'] = (1/2)**(1/T['Half Life'])

In [20]:
hl = T['Half Life']
ma = T['Max Adstock']
satr = T['Saturation']
coef = T['Coefficient']
types = T['Type']
ret_rate = T["Ret_rate"]

In [23]:
T.head()

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Ret_rate
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835,0.500000
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069,0.250000
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006,0.629961
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022,0.793701
5,Promo,0.0,7.274308e+03,0.0,0.0,0.045272,0.000000


In [39]:


## Defining Parameters
##  ????? spends is param or var
# model.sp = pyo.Param(model.i,model.j, initialize = spends)
# sp = model.sp

##defiing below params is not reqd. as they are non-changing
model.cpm = pyo.Param(model.i,model.j, initialize = cpp)
cpm = model.cpm

# model.hlf = pyo.Param(model.i,initialize = hl)
# hlf = model.hlf

model.mad = pyo.Param(model.i,initialize = ma)
mad = model.mad

model.sat = pyo.Param(model.i,initialize = satr)
sat = model.sat

model.typ = pyo.Param(model.i,initialize = types)
typ = model.typ

model.rr = pyo.Param(model.i,initialize = ret_rate)
rr = model.rr


default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any', you can suppress this warning by
explicitly specifying 'within=Any' to the Param constructor.
(deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.7/dist-packages/pyomo/core/base/indexed_component.py:645)


In [39]:
C.tail()

,1,2,3,4,5,6
Period,,,,,,
9,0.0,0.000000,0.0,0.000134,0.256921,140.83925
10,0.0,0.000188,0.0,0.000132,0.256921,0.00000
11,0.0,0.000168,0.0,0.000151,0.256921,0.00000
12,0.0,0.000136,0.0,0.000000,0.267785,0.00000
13,0.0,0.000140,0.0,0.000000,0.138880,0.00000


In [21]:
## for avoiding initial no spends terms in optimization
import numpy as np
CPP_check = np.where(C==0,0,1) 

In [22]:
## Decision variable: spends
# Spends
model.x = pyo.Var(model.i,model.j,domain = pyo.NonNegativeReals,initialize = spends)
x = model.x

#Adstock
model.ads = pyo.Var(model.i,model.j,domain=pyo.NonNegativeReals,initialize=0)
ads = model.ads




In [28]:
#Saturation
model.satu = pyo.Var(model.i,model.j,domain=pyo.NonNegativeReals,initialize=0)
satu = model.satu

In [ ]:
#Saturation

In [41]:
##Testing

model.y = pyo.Var(model.i,model.j,domain = pyo.NonNegativeReals)
y = model.y

In [44]:
model.z = pyo.Var(model.i,domain = pyo.NonNegativeReals)
z = model.z


In [ ]:
for i in range(1,len(model.typ)):
  print(model.typ[i])

Media
Media
Media
Media
Promo


In [ ]:
# def ads_func(model,i,j):

#   # i for week, j for variable
#   for j in range(1,S.shape[1]+1):
#     model.ads[1,j] = (T.loc[j,'Max Adstock']/((1+exp(-(10*T.loc[j,'Steepness']/T.loc[j,'Max Adstock'])*(T.loc[j,'Ret_rate']*ads_func(model,i,j) - T.loc[j,'Saturation']*T.loc[j,'Max Adstock'])))]

#     for i in range(1,len(S.shape[0])):
#       model.ads[i,j] = (T.loc[j,'Max Adstock']/((1+exp(-(10*T.loc[j,'Steepness']/T.loc[j,'Max Adstock'])*(T.loc[j,'Ret_rate']*ads_func(model,i,j) - T.loc[j,'Saturation']*T.loc[j,'Max Adstock'])))]

#     ads += ads_func(model)  

In [55]:
S.shape[1]

6

In [30]:
pprint(x)

Pretty printing has been turned OFF


In [29]:
S.shape[0]

13

In [32]:
#x.pprint()

In [9]:
def adstock_saturation(model,i,j):
  for j in range(1,S.shape[1]+1):
    ads[1,j] = x[1,j]/C.loc[1,j]
  
    for i in range(2,S.shape[0]+1):
      ads[i,j] = ads[i-1,j]*T.loc[j,'Ret_rate'] + x[i,j]/C.loc[i,j]
  
  return sum(sum(T.loc[j,'Max Adstock']/(1+exp(-(10*T.loc[j,'Steepness']/T.loc[j,'Max Adstock'])*(ads[i,j]-(T.loc[j,'Saturation']*T.loc[j,'Max Adstock']))))*T.loc[j,'Coefficient'] for i in range(1,S.shape[0]+1) for j in range(1,S.shape[1]+1))
  
  #return T.loc[j,'Max Adstock']/((1+exp(-(10*T.loc[j,'Steepness']/T.loc[j,'Max Adstock'])*(ads[i,j]-(T.loc[j,'Saturation']*T.loc[j,'Max Adstock'])))))    


SyntaxError: ignored

In [24]:
def adstock_saturation(model):
  for j in range(1,S.shape[1]+1):
    ads2 = pd.DataFrame()
    ads2.loc[j,1] = x[j,1]/C.loc[1,j]
  
    for i in range(2,S.shape[0]+1):
      ads[j,i] = ads[j,i-1]*T.loc[j,'Ret_rate'] + x[j,i]/C.loc[i,j]

  satu = T.loc[i,'Max Adstock']/\
(1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
(ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))*\
T.loc[i,'Coefficient'] for i in range(1,S.shape[1]+1)) for j in range(1,S.shape[0]+1)

In [25]:
C.head()

,1,2,3,4,5,6
Period,,,,,,
1,0.010282,1.0,0.000756,1.000000,1.000000,1.000000
2,0.011041,1.0,0.000773,0.000182,0.435862,1.000000
3,0.015013,1.0,0.000512,0.000169,0.338902,168.386464
4,0.017389,1.0,0.000545,0.000149,0.313671,168.386464
5,0.016280,1.0,1.000000,0.000118,0.270023,168.386464


In [29]:
T.head()

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Ret_rate
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835,0.500000
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069,0.250000
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006,0.629961
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022,0.793701
5,Promo,0.0,7.274308e+03,0.0,0.0,0.045272,0.000000


In [54]:
CPP_check

array([[1, 0, 1, 0, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1],
       [1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 1],
       [1, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0]])

In [56]:
T

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Adjustment_factor,Ret_rate
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835,4.176702e+04,0.500000
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069,1.511057e+05,0.250000
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006,3.617750e+06,0.629961
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022,5.104543e+05,0.793701
5,Promo,0.0,7.274308e+03,0.0,0.0,0.045272,0.000000e+00,0.000000
6,Media,3.8,7.525281e+02,0.6,1.5,0.251257,9.285800e-02,0.833262


In [79]:
ads2.loc[1,2] = S.loc[1,2]/C.loc[1,2]

In [99]:
## working code with everything
import numpy as np
# def adstest():
  ##Adstocking
ads2 = pd.DataFrame()
for j in range(1,S.shape[1]+1):
  #print(j)
  if T.loc[j,'Type'] == 'Media':
    ads2.loc[1,j] = S.loc[1,j]/C.loc[1,j]
    ads2.fillna(0,inplace=True)
    #print(ads2)

    for i in range(2,S.shape[0]+1):
      ads2.loc[i,j] = ads2.loc[i-1,j]*T.loc[j,'Ret_rate'] + S.loc[i,j]/C.loc[i,j]
      ads2.fillna(0,inplace=True)
      #print('media')
  else: # for promo
    for i in range(1,S.shape[0]+1): 
      ads2.loc[i,j] = S.loc[i,j]/C.loc[i,j]
      

  
  ##Saturation transformation
satu = pd.DataFrame()
for i in range(1,S.shape[1]+1):
  if T.loc[i,'Type'] == 'Media':
    for j in range(1,S.shape[0]+1):
    
      satu.loc[j,i] = T.loc[i,'Max Adstock']/\
    (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
    (ads2.loc[j,i]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))-T.loc[i,'Adjustment_factor']   #*\
    #T.loc[i,'Coefficient'] 
  
  else:
    satu[i] = ads2[i]
# ##Replacing small -ve numbers with 0
  satu[satu<0] = 0

print(ads2)    
print(satu)


prmo
prmo
prmo
prmo
prmo
prmo
prmo
prmo
prmo
prmo
prmo
prmo
prmo
                1             2             3             4            5  \
1   163533.000000  0.000000e+00  8.122073e+06  0.000000e+00     0.000000   
2   306032.500000  0.000000e+00  2.463564e+07  7.066104e+07  3551.230769   
3   371480.250000  0.000000e+00  2.956134e+07  8.092103e+07  4913.868132   
4   334416.125000  0.000000e+00  2.518284e+07  7.635885e+07  5458.923077   
5   231775.062500  0.000000e+00  1.586419e+07  6.740568e+07  6755.626374   
6   132665.531250  0.000000e+00  9.993816e+06  5.349992e+07  7274.307692   
7    77361.765625  0.000000e+00  6.295710e+06  4.246292e+07  7274.307692   
8    47078.882812  0.000000e+00  3.966049e+06  3.440765e+07  7274.307692   
9    23539.441406  0.000000e+00  2.498454e+06  2.852018e+07  7274.307692   
10   11769.720703  1.669588e+06  1.573927e+06  2.596281e+07  7274.307692   
11    5884.860352  4.738171e+06  9.915121e+05  2.192770e+07  7274.307692   
12    2942.430176  6.52

In [40]:
x.pprint()

x : Size=78, Index=x_index
    Key     : Lower : Value             : Upper : Fixed : Stale : Domain
     (1, 1) :     0 : 1681.499999999998 :  None : False : False : NonNegativeReals
     (1, 2) :     0 : 2476.099999999997 :  None : False : False : NonNegativeReals
     (1, 3) :     0 : 3279.789999999997 :  None : False : False : NonNegativeReals
     (1, 4) :     0 : 2585.389999999995 :  None : False : False : NonNegativeReals
     (1, 5) :     0 : 1051.119999999999 :  None : False : False : NonNegativeReals
     (1, 6) :     0 : 263.5999999999997 :  None : False : False : NonNegativeReals
     (1, 7) :     0 :            173.65 :  None : False : False : NonNegativeReals
     (1, 8) :     0 :            130.63 :  None : False : False : NonNegativeReals
     (1, 9) :     0 :               0.0 :  None : False : False : NonNegativeReals
    (1, 10) :     0 :               0.0 :  None : False : False : NonNegativeReals
    (1, 11) :     0 :               0.0 :  None : False : False : NonN

In [43]:
## working code with model with everything
import numpy as np
# def adstest():
  ##Adstocking
#ads2 = pd.DataFrame()
#ads2 = pd.DataFrame()
#ads2 = pd.DataFrame()
def objective_func():
  for j in range(1,S.shape[1]+1): #j for var
    #print(j)
    if T.loc[j,'Type'] == 'Media':
      ads[j,1] = (x[j,1]/C.loc[1,j])*CPP_check[0,j-1]
      

      for i in range(2,S.shape[0]+1): #i for week
        ads[j,i] = ads[j,i-1]*T.loc[j,'Ret_rate'] + (x[j,i]/C.loc[i,j])*CPP_check[i-1,j-1]
        
    else: # for promo
      for i in range(1,S.shape[0]+1): 
        ads[j,i] = (S.loc[i,j]/C.loc[i,j])*CPP_check[i-1,j-1]
        

    
    ##Saturation transformation
  #satu = pd.DataFrame()
  for i in range(1,S.shape[1]+1): #j for week, i for var
    if T.loc[i,'Type'] == 'Media':
      for j in range(1,S.shape[0]+1):
      
        satu[i,j] = (T.loc[i,'Max Adstock']/\
      (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
      (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))-T.loc[i,'Adjustment_factor'])*T.loc[i,'Coefficient']   #*\
      #T.loc[i,'Coefficient'] 
    
    else: 
      for j in range(1,S.shape[0]+1): # j for week
        satu[i,j] = ads[i,j]
      

  return sum(sum(sat[i,j] for i in range(1,S.shape[1]+1)) for j in range(1,S.shape[0]+1))

# ##Replacing small -ve numbers with 0
  #satu[satu<0] = 0



In [52]:
model.x.value

1681.499999999998

In [57]:
S.head()

,1,2,3,4,5,6
Period,,,,,,
1,1681.50,0.0,6141.681550,0.00,0.000000,0.000000
2,2476.10,0.0,15080.853226,12833.30,1547.846154,0.000000
3,3279.79,0.0,7192.357366,4191.13,1665.318681,39133.014118
4,2585.39,0.0,3574.155648,1806.90,1712.307692,39133.014118
5,1051.12,0.0,0.000000,800.00,1824.175824,5590.430588


In [66]:
S.shape[1]

6

In [68]:
#Spends sum for constraints
spends_sum = []
for i in range(S.shape[1]):
  spends_sum.append(sum(S[S.columns[i]]))
  print(sum(S[S.columns[i]]))

11641.779999999986
2713.2644939999964
31989.04778999995
20531.32999999
20155.63736263736
172021.8294117647


In [69]:
spends_sum

[11641.779999999986, 2713.2644939999964, 31989.04778999995, 20531.32999999, 20155.63736263736, 172021.8294117647]

In [63]:
sum(S[S.columns[0]])

11641.779999999986

In [ ]:
sum(x[i] for i in range)

In [70]:
x[1:3,1:3]

IndexError: ignored

In [71]:
S.shape[1]

6

In [ ]:
##Constraints

def Constraint1(model,i,j):
  return sum(x[1,j] for j in range(1,S.shape[0]+1)) <= spends_sum[0]
model.Const1 = pyo.Constraint(x, rule = Constraint1)

def Constraint2(model,i,j):
  return sum(x[2,j] for j in range(1,S.shape[0]+1)) <= spends_sum[1]
model.Const2 = pyo.Constraint(x, rule = Constraint2)


def Constraint3(model,i,j):
  return sum(x[3,j] for j in range(1,S.shape[0]+1)) <= spends_sum[2]
model.Const3 = pyo.Constraint(x, rule = Constraint3)


def Constraint4(model,i,j):
  return sum(x[4,j] for j in range(1,S.shape[0]+1)) <= spends_sum[3]
model.Const4 = pyo.Constraint(x, rule = Constraint4)


def Constraint5(model,i,j):
  return sum(x[5,j] for j in range(1,S.shape[0]+1)) <= spends_sum[4]
model.Const5 = pyo.Constraint(x, rule = Constraint5)


def Constraint6(model,i,j):
  return sum(x[6,j] for j in range(1,S.shape[0]+1)) <= spends_sum[5]
model.Const6 = pyo.Constraint(x, rule = Constraint6)


def Constraint1(model,i,j):
  return sum(sum(x[i,j] for j in range(1,S.shape[0]+1)) for j in range(1,S.shape[1]+1)) <= spends_sum[1]
model.Const2 = pyo.Constraint(x, rule = Constraint2)

In [46]:
# for i in model.x:
#   print(i)

In [26]:
## working code with everything
import numpy as np
# def adstest():
  ##Adstocking
ads2 = pd.DataFrame()
for j in range(1,S.shape[1]+1):
  #print(j)
  if T.loc[j,'Type'] == 'Media':
    ads2.loc[1,j] = (S.loc[1,j]/C.loc[1,j])*CPP_check[i-1,j-1]
    #ads2.fillna(0,inplace=True)
    #print(ads2)

    for i in range(2,S.shape[0]+1):
      ads2.loc[i,j] = ads2.loc[i-1,j]*T.loc[j,'Ret_rate'] + (S.loc[i,j]/C.loc[i,j])*CPP_check[i-1,j-1]
      #ads2.fillna(0,inplace=True)
      #print('media')
  else: # for promo
    for i in range(1,S.shape[0]+1): 
      ads2.loc[i,j] = (S.loc[i,j]/C.loc[i,j])*CPP_check[i-1,j-1]
      

  
  ##Saturation transformation
satu = pd.DataFrame()
for i in range(1,S.shape[1]+1):
  if T.loc[i,'Type'] == 'Media':
    for j in range(1,S.shape[0]+1):
    
      satu.loc[j,i] = (T.loc[i,'Max Adstock']/\
    (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
    (ads2.loc[j,i]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))-T.loc[i,'Adjustment_factor'])*T.loc[i,'Coefficient']   #*\
    #T.loc[i,'Coefficient'] 
  
  else:
    satu[i] = ads2[i]
# ##Replacing small -ve numbers with 0
  satu[satu<0] = 0

print(ads2)    
print(satu)


                1             2             3             4            5  \
1   163533.000000  0.000000e+00  8.122073e+06  0.000000e+00     0.000000   
2   306032.500000  0.000000e+00  2.463564e+07  7.066104e+07  3551.230769   
3   371480.250000  0.000000e+00  2.956134e+07  8.092103e+07  4913.868132   
4   334416.125000  0.000000e+00  2.518284e+07  7.635885e+07  5458.923077   
5   231775.062500  0.000000e+00  1.586419e+07  6.740568e+07  6755.626374   
6   132665.531250  0.000000e+00  9.993816e+06  5.349992e+07  7274.307692   
7    77361.765625  0.000000e+00  6.295710e+06  4.246292e+07  7274.307692   
8    47078.882812  0.000000e+00  3.966049e+06  3.440765e+07  7274.307692   
9    23539.441406  0.000000e+00  2.498454e+06  2.852018e+07  7274.307692   
10   11769.720703  1.669588e+06  1.573927e+06  2.596281e+07  7274.307692   
11    5884.860352  4.738171e+06  9.915121e+05  2.192770e+07  7274.307692   
12    2942.430176  6.521022e+06  6.246135e+05  1.740403e+07  6140.241758   
13    1471.2

In [27]:
adstest()

NameError: ignored

In [ ]:
import numpy as np
def adstest():
  ads2 = pd.DataFrame()
  for j in range(1,S.shape[1]+1):
    ads2.loc[1,j] = S.loc[1,j]/C.loc[1,j]
    ads2.fillna(0,inplace=True)

    for i in range(2,S.shape[0]+1):
      ads2.loc[i,j] = ads2.loc[i-1,j]*T.loc[j,'Ret_rate'] + S.loc[i,j]/C.loc[i,j]
      ads2.fillna(0,inplace=True)
  return ads2    


In [153]:
ads2 = pd.DataFrame()
for j in range(1,S.shape[1]+1):
  #ads2.loc[1,j] = S.loc[1,j]/C.loc[1,j]
  print(S.loc[1,j])
  print(C.loc[1,j])

1681.499999999998
0.010282328337399779
0.0
0.0
6141.681549999989
0.0007561716756300995
0.0
0.0
0.0
0.0
0.0
0.0


In [103]:
for j in range(1,S.shape[0]+1):
  for i in range(1,S.shape[1]+1):

    a = sum(sum(T.loc[i,'Max Adstock']/\
    (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
    (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))*\
    T.loc[i,'Coefficient'] for i in range(1,S.shape[1]+1)) for j in range(1,S.shape[0]+1))

In [112]:
##Final code ready

sum(sum(T.loc[i,'Max Adstock']/\
    (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
    (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))*\
    T.loc[i,'Coefficient'] for i in range(1,S.shape[1]+1)) for j in range(1,S.shape[0]+1))

In [106]:
fo i in range(a.values


AttributeError: ignored

In [100]:
for i in model.i:
  print(i)

1
2
3
4
5
6


In [108]:
ads[1,1].value

0

In [110]:
a[1,1]

TypeError: ignored

In [94]:
a = sum(sum(i) for i in range(1,5)) for j in range(1,3) 

SyntaxError: ignored

In [89]:
S.shape[0]

13

In [90]:
for i in range(1,S.shape[1]+1):
  for j in range(1,S.shape[0]+1):

    T.loc[i,'Max Adstock']/\
        (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
        (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))*\
        T.loc[i,'Coefficient']

In [93]:
sum(sum(T.loc[i,'Max Adstock']/\
        (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
        (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))*\
        T.loc[i,'Coefficient']) for j in range(1,S.shape[0]+1)) for i in range(1,S.shape[1]+1)

TypeError: ignored

In [91]:
T.head()

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Ret_rate
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835,0.500000
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069,0.250000
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006,0.629961
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022,0.793701
5,Promo,0.0,7.274308e+03,0.0,0.0,0.045272,0.000000


In [47]:
# for i in ads:
#   print(ads[i],ads[i].value)

In [24]:
S.head()

,1,2,3,4,5,6
Period,,,,,,
1,1681.50,0.0,6141.681550,0.00,0.000000,0.000000
2,2476.10,0.0,15080.853226,12833.30,1547.846154,0.000000
3,3279.79,0.0,7192.357366,4191.13,1665.318681,39133.014118
4,2585.39,0.0,3574.155648,1806.90,1712.307692,39133.014118
5,1051.12,0.0,0.000000,800.00,1824.175824,5590.430588


In [26]:
T.head(8)

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Ret_rate
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835,0.500000
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069,0.250000
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006,0.629961
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022,0.793701
5,Promo,0.0,7.274308e+03,0.0,0.0,0.045272,0.000000
6,Media,3.8,7.525281e+02,0.6,1.5,0.251257,0.833262


In [ ]:
## Objective Function
def objective_rule(model):
  for i in range(1,len(model.typ)+1): ##i for var
    if model.typ[i] == "Media":
      


0.0

In [ ]:

from pyomo.environ import *

CPP = Spends/Raw Execution(RE)

RE = Spends/CPP

Adstocked execution = RE * Adstock factor(AF)
= (Spends/CPP)*AF

Avg adst execution = (Spends*AF/CPP)/Exec. periods

Now apply transformation on Avg adst execution to get TCP

Lift = TCP * Exec. periods * coeffcient

Put lift in Optimization equation



In [ ]:
## Defining the model
import numpy as np
#import math
from pyomo.environ import *
model = pyo.ConcreteModel()


sp1 = 33150.8411764706
sp2 = 233150.268146738


## Decision Variables
model.s1 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.9*sp1,1.1*sp1)) 
s1 = model.s1
model.s2 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.9*sp2,1.1*sp2))
s2 = model.s2


expre = ((1953.64352941177/(1+exp(-(10*3/1953.64352941177)*((s1*1/47/1)-0.179872867132035*1953.64352941177))))-8.81752993140885)*0.00133831972924945*47 + ((11441.0307432077/(1+exp(-(10*1/11441.0307432077)*((s2*1/1/154)-0.202998593210211*11441.0307432077))))-1328.19344200206)*0.00245568243410747*154

## Objective Function
model.Obj = pyo.Objective(expr =expre, sense = pyo.maximize)

## Constraint
model.Const1 = pyo.Constraint(expr = s1+s2<=266301.109323209)



optm = SolverFactory('couenne',executable='/content/couenne')
results = optm.solve(model)



print('objective function =', model.Obj())
print('s1 = ',model.s1())
print('s2 = ',model.s2())

print('sum total = ',model.s1()+model.s2())

objective function = 1065.7698788838547
s1 =  29835.757357158283
s2 =  236465.35226438538
sum total =  266301.10962154367


In [ ]:
# model.s3 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp3,1.3*sp3))
# s3 = model.s3
# model.s4 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp4,1.3*sp4))
# s4 = model.s4
# model.s5 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp5,1.3*sp5))
# s5 = model.s5
# model.s6 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp6,1.3*sp6))
# s6 = model.s6